### Using SMIRNOFF in host-guest systems

This is a refresh of a notebook written by David Mobley

https://github.com/openforcefield/openff-toolkit/blob/master/examples/deprecated/host_guest_simulation/smirnoff_host_guest.ipynb

In [1]:
import mdtraj as md
import nglview
from openeye import oechem, oedocking

assert oechem.OEChemIsLicensed()

from openff.toolkit.topology import Molecule, Topology
from openff.toolkit.typing.engines.smirnoff import ForceField

from openff.interchange.components.interchange import Interchange

In [2]:
guest_smiles = "OC(CC1CCCC1)=O"

guest = Molecule.from_smiles(guest_smiles)
guest.assign_partial_charges(partial_charge_method="am1bcc")
guest.generate_conformers(n_conformers=100)
guest

NGLWidget(max_frame=5)

In [3]:
host = Molecule.from_file("OA.mol2")
host

NGLWidget()

In [4]:
oe_guest = guest.to_openeye()
oe_host = host.to_openeye()

center_of_mass = oechem.OEFloatArray(3)
oechem.OEGetCenterOfMass(oe_host, center_of_mass)

# Create receptor, as per https://docs.eyesopen.com/toolkits/python/dockingtk/receptor.html#creating-a-receptor
receptor = oechem.OEGraphMol()
oedocking.OEMakeReceptor(receptor, host.to_openeye(), *center_of_mass)

dock = oedocking.OEDock()
dock.Initialize(receptor)

dockedMol = oechem.OEGraphMol()
status = dock.DockMultiConformerMolecule(dockedMol, oe_guest)
sdtag = oedocking.OEDockMethodGetName(oedocking.OEDockMethod_Chemgauss4)
oedocking.OESetSDScore(dockedMol, dock, sdtag)
dock.AnnotatePose(dockedMol)

# Write out docked pose if docking successful
if status == oedocking.OEDockingReturnCode_Success:
    outmol = dockedMol

    # Write out
    tripos_mol2_filename = "docked_guest.mol2"
    ofile = oechem.oemolostream(tripos_mol2_filename)
    oechem.OEWriteMolecule(ofile, outmol)
    ofile.close()

    # Clean up residue names in mol2 files that are tleap-incompatible: replace substructure names with valid text.
    infile = open(tripos_mol2_filename)
    lines = infile.readlines()
    infile.close()
    newlines = [line.replace("<0>", "GUEST") for line in lines]
    outfile = open(tripos_mol2_filename, "w")
    outfile.writelines(newlines)
    outfile.close()
else:
    raise Exception("Error: Docking failed.")

In [5]:
nglview.show_mdtraj(md.load("OA.mol2").stack(md.load("docked_guest.mol2")))

NGLWidget()

In [6]:
_host = md.load("OA.mol2")
_guest = md.load("docked_guest.mol2")

# Figure out which atom indices correspond to the guest, for use in visualization
atoms_guest = [_host.n_atoms + i for i in range(_guest.n_atoms)]

# "Stack" host and guest Trajectory objects into a single object
combined = _host.stack(_guest)

# Visualize
view = nglview.show_mdtraj(combined)
view.add_representation("spacefill", selection="all")
view.add_representation("spacefill", selection=atoms_guest, color="green")
view

NGLWidget()

In [7]:
docked_topology = Topology.from_molecules(
    [Molecule.from_openeye(oemol) for oemol in [oe_host, oe_guest]]
)

In [8]:
sage = ForceField("openff-2.0.0.offxml")

In [9]:
docked = Interchange.from_smirnoff(sage, docked_topology)

Warning (not error because allow_undefined_stereo=True): Unable to make OFFMol from RDMol: RDMol has unspecified stereochemistry. RDMol name: OCTUndefined chiral centers are:
 - Atom C (index 7)
 - Atom C (index 8)
 - Atom C (index 17)
 - Atom C (index 20)
 - Atom C (index 23)
 - Atom C (index 33)
 - Atom C (index 37)
 - Atom C (index 39)

/Users/mwt/miniconda3/envs/interchange-topology/bin/wrapped_progs/antechamber: Fatal Error!
Cannot properly run "/Users/mwt/miniconda3/envs/interchange-topology/bin/sqm -O -i sqm.in -o sqm.out".


ValueError: No registered toolkits can provide the capability "assign_partial_charges" for args "()" and kwargs "{'molecule': Molecule with name 'OCT' and SMILES '[H]c1c(c(c2c(c1Oc3c(c4c(c(c3[H])Oc5c(c(c(c(c5[H])Oc6c(c7c(c(c6[H])Oc8c(c(c(c(c8[H])Oc9c(c1c(c(c9[H])Oc3c(c(c(c(c3[H])Oc3c(c(c(c(c3[H])O2)[H])C2(Oc3c(c5c6c(c3C(c3c(c8c(c(c3O2)[H])OC1(Oc1c(c2c(c(c1C8([H])C([H])([H])C([H])([H])[C-]([O-])[O-])[H])C(c1c(c(c(c(c1OC7(O2)[H])[H])OC4(O5)[H])C6([H])C([H])([H])C([H])([H])[C-]([O-])[O-])[H])([H])C([H])([H])C([H])([H])[C-]([O-])[O-])[H])[H])[H])([H])C([H])([H])C([H])([H])[C-]([O-])[O-])[H])[H])[H])[H])[H])[C-]([O-])[O-])[H])[H])[H])[H])[C-]([O-])[O-])[H])[H])[H])[H])[C-]([O-])[O-])[H])[H])[H])[H])[H])[C-]([O-])[O-]', 'partial_charge_method': 'am1bcc', 'use_conformers': None, 'strict_n_conformers': False, 'normalize_partial_charges': True, '_cls': <class 'openff.toolkit.topology.molecule.Molecule'>}"
Available toolkits are: [ToolkitWrapper around OpenEye Toolkit version 2021.1.1, ToolkitWr